In [1]:
import torch 
import torch.nn.functional as F
from torch import nn
import torch
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger



wandb_logger = WandbLogger(project="LAB5")


/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: oeg1n18. Use `wandb login --relogin` to force relogin


# An initial attempt

In [2]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

    def __getitem__(self, index):
        if index >= len(self):
            raise IndexError("{} index out of range".format(self.__class__.__name__))

        rng_state = torch.get_rng_state()
        torch.manual_seed(index + self.random_offset)

        while True:
            img = torch.zeros(self.dim, self.dim)
            dx = torch.randint(-10,10,(1,),dtype=torch.float)
            dy = torch.randint(-10,10,(1,),dtype=torch.float)
            c = torch.randint(-20,20,(1,), dtype=torch.float)

            params = torch.cat((dy/dx, c))
            xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
            xy[:,1] = xy[:,0] * params[0] + params[1]

            xy.round_()
            xy = xy[ xy[:,1] > 0 ]
            xy = xy[ xy[:,1] < self.dim ]
            xy = xy[ xy[:,0] < self.dim ]

            for i in range(xy.shape[0]):
                x, y = xy[i][0], self.dim - xy[i][1]
                img[int(y), int(x)]=1
            if img.sum() > 2:
                break

        torch.set_rng_state(rng_state)
        return img.unsqueeze(0), params

    def __len__(self):
        return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

In [3]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=128)
val_loader = DataLoader(val_data, shuffle=True, batch_size=128)
test_loader = DataLoader(test_data, shuffle=True, batch_size=128)

In [4]:
class simpleModel(nn.Module):
    def __init__(self):
        super(simpleModel, self).__init__()
        self.conv1  = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(48*40*40, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [5]:
# define the LightningModule
class train_module(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        y_hat = self.model(x)
        loss = nn.functional.mse_loss(y_hat, y)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss",loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        y_hat = self.model(x)
        val_loss = F.mse_loss(y_hat, y)
        self.log("val_loss", val_loss)

In [6]:
model = simpleModel()
module = train_module(model)

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/ollie/miniconda3/envs/cv_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/ollie/miniconda3/envs/cv_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [7]:
trainer = pl.Trainer(max_epochs=100, accelerator="gpu", logger=wandb_logger)
trainer.fit(model=module, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | simpleModel | 9.8 M 
--------------------------------------
9.8 M     Trainable params
0         Non-trainable params
9.8 M     Total params
39.325    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|                       | 0/2 [00:00<?, ?it/s]

/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  91%|██████████ | 40/44 [00:01<00:00, 25.43it/s, loss=56.8, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 1:  91%|██████████ | 40/44 [00:01<00:00, 26.15it/s, loss=41.8, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 2:  91%|██████████ | 40/44 [00:01<00:00, 25.75it/s, loss=28.2, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 3:  91%|██████████ | 40/44 [00:01<00:00, 25.38it/s, loss=22.6, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 4:  91%|██████████ | 40/44 [00:01<00:00, 26.31it/s, loss=20.4, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 5:  91%|██████████ | 40/44 [00:01<00:00, 26.39it/s, loss=18.8, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 6:  91%|██████████ | 40/44 [00:01<00:00, 26.10it/s, loss=21.4, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 7:  91%|██████████ | 40/44 [00:01<00:00, 26.05it/s, loss=17.3, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 8:  91%|██████████ | 40/44 [00:01<00:00, 25.74it/s, loss=17.9, v_num=7lln]
Validation: 0it [00:00,

...

Epoch 66:  91%|█████████ | 40/44 [00:01<00:00, 26.73it/s, loss=1.77, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 67:  91%|█████████ | 40/44 [00:01<00:00, 26.76it/s, loss=2.29, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 68:  91%|██████████ | 40/44 [00:01<00:00, 26.62it/s, loss=1.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 69:  91%|█████████ | 40/44 [00:01<00:00, 26.55it/s, loss=1.67, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 70:  91%|█████████ | 40/44 [00:01<00:00, 26.46it/s, loss=1.45, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 71:  91%|█████████ | 40/44 [00:01<00:00, 26.83it/s, loss=1.44, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 72:  91%|█████████ | 40/44 [00:01<00:00, 26.47it/s, loss=1.59, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 73:  91%|█████████ | 40/44 [00:01<00:00, 25.96it/s, loss=1.47, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 74:  91%|█████████ | 40/44 [00:01<00:00, 26.15it/s, loss=1.22, v_num=7lln]
Validation: 0it [00:00,

Epoch 79:  91%|█████████ | 40/44 [00:01<00:00, 26.24it/s, loss=1.12, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 80:  91%|█████████ | 40/44 [00:01<00:00, 26.26it/s, loss=1.13, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 81:  91%|█████████ | 40/44 [00:01<00:00, 26.06it/s, loss=1.17, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 82:  91%|█████████ | 40/44 [00:01<00:00, 26.05it/s, loss=1.02, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 83:  91%|████████▏| 40/44 [00:01<00:00, 26.14it/s, loss=0.985, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 84:  91%|█████████ | 40/44 [00:01<00:00, 26.06it/s, loss=1.02, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 85:  91%|█████████ | 40/44 [00:01<00:00, 26.36it/s, loss=1.01, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 86:  91%|█████████ | 40/44 [00:01<00:00, 26.08it/s, loss=1.21, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 87:  91%|████████▏| 40/44 [00:01<00:00, 25.48it/s, loss=0.957, v_num=7lln]
Validation: 0it [00:00,

Epoch 92:  91%|█████████ | 40/44 [00:01<00:00, 25.82it/s, loss=0.85, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 93:  91%|██████████ | 40/44 [00:01<00:00, 25.76it/s, loss=0.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 94:  91%|████████▏| 40/44 [00:01<00:00, 26.39it/s, loss=0.669, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 95:  91%|████████▏| 40/44 [00:01<00:00, 26.36it/s, loss=0.823, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 96:  91%|████████▏| 40/44 [00:01<00:00, 26.29it/s, loss=0.681, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 97:  91%|█████████ | 40/44 [00:01<00:00, 26.45it/s, loss=0.66, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 98:  91%|████████▏| 40/44 [00:01<00:00, 26.41it/s, loss=0.601, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99:  91%|████████▏| 40/44 [00:01<00:00, 26.50it/s, loss=0.559, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99: 100%|█████████| 44/44 [00:01<00:00, 26.49it/s, loss=0.559, v_num=7lln]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████| 44/44 [00:01<00:00, 24.33it/s, loss=0.559, v_num=7lln]


# A Second Attempt

In [8]:
class simpleModel(nn.Module):
    def __init__(self):
        super(simpleModel, self).__init__()
        self.conv1  = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1)
        self.conv2 = nn.Conv2d(48, 48, (3,3), stride=1, padding=1)
        self.pool = nn.AdaptiveMaxPool2d((25,25))
        self.fc1 = nn.Linear(48*25*25, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
model = simpleModel()
module = train_module(model)
trainer = pl.Trainer(max_epochs=100, accelerator="gpu", logger=wandb_logger)
trainer.fit(model=module, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ollie/miniconda3/envs/cv_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory ./LAB5/ypsp7lln/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | simpleModel | 3.9 M 
--------------------------------------
3.9 M     Traina

Epoch 0:  91%|██████████ | 40/44 [00:01<00:00, 25.16it/s, loss=37.5, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 1:  91%|██████████ | 40/44 [00:01<00:00, 25.16it/s, loss=21.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 2:  91%|██████████ | 40/44 [00:01<00:00, 25.26it/s, loss=14.5, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 3:  91%|██████████ | 40/44 [00:01<00:00, 25.26it/s, loss=9.65, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 4:  91%|██████████ | 40/44 [00:01<00:00, 25.20it/s, loss=7.21, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 5:  91%|██████████ | 40/44 [00:01<00:00, 24.84it/s, loss=5.93, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 6:  91%|██████████ | 40/44 [00:01<00:00, 24.51it/s, loss=4.81, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 7:  91%|██████████▉ | 40/44 [00:01<00:00, 25.36it/s, loss=3.9, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 8:  91%|██████████ | 40/44 [00:01<00:00, 25.33it/s, loss=3.27, v_num=7lln]
Validation: 0it [00:00,

Validation: 0it [00:00, ?it/s]
Epoch 14:  91%|█████████ | 40/44 [00:01<00:00, 25.02it/s, loss=1.62, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 15:  91%|█████████ | 40/44 [00:01<00:00, 25.15it/s, loss=1.32, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 16:  91%|█████████ | 40/44 [00:01<00:00, 25.00it/s, loss=1.37, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 17:  91%|█████████ | 40/44 [00:01<00:00, 25.09it/s, loss=1.22, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 18:  91%|█████████ | 40/44 [00:01<00:00, 25.09it/s, loss=1.27, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 19:  91%|████████▏| 40/44 [00:01<00:00, 25.04it/s, loss=0.974, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 20:  91%|████████▏| 40/44 [00:01<00:00, 25.06it/s, loss=0.839, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 21:  91%|████████▏| 40/44 [00:01<00:00, 25.07it/s, loss=0.895, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 22:  91%|████████▏| 40/44 [00:01<00:00, 25.42it/s, loss=0.769, v_nu

Epoch 27:  91%|█████████ | 40/44 [00:01<00:00, 24.90it/s, loss=0.54, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 28:  91%|████████▏| 40/44 [00:01<00:00, 24.84it/s, loss=0.438, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 29:  91%|████████▏| 40/44 [00:01<00:00, 24.98it/s, loss=0.355, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 30:  91%|████████▏| 40/44 [00:01<00:00, 24.87it/s, loss=0.468, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 31:  91%|█████████ | 40/44 [00:01<00:00, 25.06it/s, loss=0.41, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 32:  91%|████████▏| 40/44 [00:01<00:00, 24.31it/s, loss=0.353, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 33:  91%|█████████ | 40/44 [00:01<00:00, 24.70it/s, loss=0.38, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 34:  91%|█████████ | 40/44 [00:01<00:00, 24.69it/s, loss=0.37, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 35:  91%|████████▏| 40/44 [00:01<00:00, 25.10it/s, loss=0.299, v_num=7lln]
Validation: 0it [00:00,

Epoch 40:  91%|████████▏| 40/44 [00:01<00:00, 24.96it/s, loss=0.238, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 41:  91%|████████▏| 40/44 [00:01<00:00, 25.06it/s, loss=0.218, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 42:  91%|████████▏| 40/44 [00:01<00:00, 24.65it/s, loss=0.192, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 43:  91%|████████▏| 40/44 [00:01<00:00, 24.71it/s, loss=0.155, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 44:  91%|████████▏| 40/44 [00:01<00:00, 24.76it/s, loss=0.238, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 45:  91%|████████▏| 40/44 [00:01<00:00, 24.34it/s, loss=0.166, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 46:  91%|████████▏| 40/44 [00:01<00:00, 24.55it/s, loss=0.169, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 47:  91%|█████████ | 40/44 [00:01<00:00, 25.02it/s, loss=0.16, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 48:  91%|████████▏| 40/44 [00:01<00:00, 25.11it/s, loss=0.149, v_num=7lln]
Validation: 0it [00:00,

...

Epoch 66:  91%|███████▎| 40/44 [00:01<00:00, 25.00it/s, loss=0.0762, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 67:  91%|███████▎| 40/44 [00:01<00:00, 24.70it/s, loss=0.0752, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 68:  91%|███████▎| 40/44 [00:01<00:00, 24.66it/s, loss=0.0687, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 69:  91%|███████▎| 40/44 [00:01<00:00, 24.92it/s, loss=0.0964, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 70:  91%|███████▎| 40/44 [00:01<00:00, 25.03it/s, loss=0.0806, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 71:  91%|███████▎| 40/44 [00:01<00:00, 24.82it/s, loss=0.0885, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 72:  91%|████████▏| 40/44 [00:01<00:00, 24.93it/s, loss=0.082, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 73:  91%|███████▎| 40/44 [00:01<00:00, 24.96it/s, loss=0.0968, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 74:  91%|████████▏| 40/44 [00:01<00:00, 25.25it/s, loss=0.129, v_num=7lln]
Validation: 0it [00:00,

Epoch 79:  91%|████████▏| 40/44 [00:01<00:00, 24.88it/s, loss=0.123, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 80:  91%|████████▏| 40/44 [00:01<00:00, 24.92it/s, loss=0.121, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 81:  91%|████████▏| 40/44 [00:01<00:00, 24.99it/s, loss=0.124, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 82:  91%|███████▎| 40/44 [00:01<00:00, 24.91it/s, loss=0.0885, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 83:  91%|███████▎| 40/44 [00:01<00:00, 25.27it/s, loss=0.0906, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 84:  91%|███████▎| 40/44 [00:01<00:00, 25.31it/s, loss=0.0744, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 85:  91%|████████▏| 40/44 [00:01<00:00, 25.03it/s, loss=0.101, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 86:  91%|████████▏| 40/44 [00:01<00:00, 24.89it/s, loss=0.274, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 87:  91%|████████▏| 40/44 [00:01<00:00, 24.98it/s, loss=0.117, v_num=7lln]
Validation: 0it [00:00,

Epoch 92:  91%|███████▎| 40/44 [00:01<00:00, 24.70it/s, loss=0.0344, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 93:  91%|███████▎| 40/44 [00:01<00:00, 24.79it/s, loss=0.0447, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 94:  91%|███████▎| 40/44 [00:01<00:00, 24.84it/s, loss=0.0403, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 95:  91%|████████▏| 40/44 [00:01<00:00, 25.29it/s, loss=0.043, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 96:  91%|███████▎| 40/44 [00:01<00:00, 25.07it/s, loss=0.0309, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 97:  91%|███████▎| 40/44 [00:01<00:00, 25.42it/s, loss=0.0271, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 98:  91%|███████▎| 40/44 [00:01<00:00, 25.52it/s, loss=0.0258, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99:  91%|███████▎| 40/44 [00:01<00:00, 25.42it/s, loss=0.0297, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99: 100%|████████| 44/44 [00:01<00:00, 25.53it/s, loss=0.0297, v_num=7lln]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|████████| 44/44 [00:01<00:00, 24.47it/s, loss=0.0297, v_num=7lln]


# Something that actually Works

In [10]:
device = "cuda"

class simpleModel(nn.Module):
    def __init__(self):
        super(simpleModel, self).__init__()
        self.conv1  = nn.Conv2d(3, 48, (3, 3), stride=1, padding=1)
        self.conv2 = nn.Conv2d(48, 48, (3,3), stride=1, padding=1)
        self.pool = nn.AdaptiveMaxPool2d((25,25))
        self.fc1 = nn.Linear(48*25*25, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        idxx = torch.repeat_interleave(torch.arange(-20, 20, dtype=torch.float).unsqueeze(0)/40.0, 
                                     repeats=40, dim=0).to(device)
        idxy = idxx.clone().t()
        idx = torch.stack([idxx, idxy]).unsqueeze(0)
        idx = torch.repeat_interleave(idx, repeats=x.shape[0], dim=0)
        x = torch.cat([x, idx], dim=1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
model = simpleModel()
module = train_module(model)
trainer = pl.Trainer(max_epochs=100, accelerator="gpu", logger=wandb_logger)
trainer.fit(model=module, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | simpleModel | 3.9 M 
--------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.450    Total estimated model params size (MB)


Epoch 0:  91%|██████████ | 40/44 [00:01<00:00, 25.33it/s, loss=38.2, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 1:  91%|██████████ | 40/44 [00:01<00:00, 25.43it/s, loss=22.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 2:  91%|██████████ | 40/44 [00:01<00:00, 25.41it/s, loss=19.2, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 3:  91%|██████████ | 40/44 [00:01<00:00, 25.43it/s, loss=15.3, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 4:  91%|██████████ | 40/44 [00:01<00:00, 25.42it/s, loss=14.4, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 5:  91%|██████████ | 40/44 [00:01<00:00, 25.30it/s, loss=10.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 6:  91%|██████████ | 40/44 [00:01<00:00, 25.21it/s, loss=10.3, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 7:  91%|██████████ | 40/44 [00:01<00:00, 25.19it/s, loss=8.77, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 8:  91%|██████████ | 40/44 [00:01<00:00, 25.19it/s, loss=8.01, v_num=7lln]
Validation: 0it [00:00,

Validation: 0it [00:00, ?it/s]
Epoch 14:  91%|█████████ | 40/44 [00:01<00:00, 25.34it/s, loss=5.14, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 15:  91%|█████████ | 40/44 [00:01<00:00, 25.52it/s, loss=5.08, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 16:  91%|█████████ | 40/44 [00:01<00:00, 25.53it/s, loss=4.77, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 17:  91%|█████████ | 40/44 [00:01<00:00, 25.49it/s, loss=3.68, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 18:  91%|██████████ | 40/44 [00:01<00:00, 25.34it/s, loss=3.6, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 19:  91%|██████████ | 40/44 [00:01<00:00, 25.25it/s, loss=3.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 20:  91%|██████████ | 40/44 [00:01<00:00, 25.46it/s, loss=3.5, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 21:  91%|█████████ | 40/44 [00:01<00:00, 25.48it/s, loss=2.89, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 22:  91%|█████████ | 40/44 [00:01<00:00, 25.50it/s, loss=2.89, v_nu

Epoch 27:  91%|█████████ | 40/44 [00:01<00:00, 25.45it/s, loss=2.32, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 28:  91%|█████████ | 40/44 [00:01<00:00, 25.51it/s, loss=2.45, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 29:  91%|█████████ | 40/44 [00:01<00:00, 25.47it/s, loss=1.93, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 30:  91%|█████████ | 40/44 [00:01<00:00, 25.42it/s, loss=1.98, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 31:  91%|█████████ | 40/44 [00:01<00:00, 25.44it/s, loss=1.88, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 32:  91%|█████████ | 40/44 [00:01<00:00, 25.26it/s, loss=1.91, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 33:  91%|█████████ | 40/44 [00:01<00:00, 25.53it/s, loss=1.81, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 34:  91%|██████████ | 40/44 [00:01<00:00, 25.45it/s, loss=1.7, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 35:  91%|█████████ | 40/44 [00:01<00:00, 25.47it/s, loss=1.58, v_num=7lln]
Validation: 0it [00:00,

Epoch 40:  91%|█████████ | 40/44 [00:01<00:00, 25.48it/s, loss=1.51, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 41:  91%|█████████ | 40/44 [00:01<00:00, 25.44it/s, loss=1.19, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 42:  91%|█████████ | 40/44 [00:01<00:00, 25.48it/s, loss=1.13, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 43:  91%|█████████ | 40/44 [00:01<00:00, 25.49it/s, loss=1.04, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 44:  91%|█████████ | 40/44 [00:01<00:00, 25.38it/s, loss=1.07, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 45:  91%|█████████ | 40/44 [00:01<00:00, 25.37it/s, loss=1.32, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 46:  91%|█████████ | 40/44 [00:01<00:00, 25.44it/s, loss=1.03, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 47:  91%|████████▏| 40/44 [00:01<00:00, 25.52it/s, loss=0.986, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 48:  91%|████████▏| 40/44 [00:01<00:00, 25.49it/s, loss=0.874, v_num=7lln]
Validation: 0it [00:00,

...

Epoch 66:  91%|████████▏| 40/44 [00:01<00:00, 25.36it/s, loss=0.687, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 67:  91%|████████▏| 40/44 [00:01<00:00, 25.23it/s, loss=0.478, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 68:  91%|████████▏| 40/44 [00:01<00:00, 25.27it/s, loss=0.497, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 69:  91%|████████▏| 40/44 [00:01<00:00, 25.34it/s, loss=0.557, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 70:  91%|████████▏| 40/44 [00:01<00:00, 25.33it/s, loss=0.467, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 71:  91%|████████▏| 40/44 [00:01<00:00, 25.38it/s, loss=0.437, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 72:  91%|████████▏| 40/44 [00:01<00:00, 25.34it/s, loss=0.446, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 73:  91%|████████▏| 40/44 [00:01<00:00, 25.29it/s, loss=0.453, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 74:  91%|█████████ | 40/44 [00:01<00:00, 25.33it/s, loss=0.35, v_num=7lln]
Validation: 0it [00:00,

Epoch 79:  91%|████████▏| 40/44 [00:01<00:00, 25.42it/s, loss=0.401, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 80:  91%|█████████ | 40/44 [00:01<00:00, 25.04it/s, loss=0.32, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 81:  91%|████████▏| 40/44 [00:01<00:00, 25.25it/s, loss=0.259, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 82:  91%|████████▏| 40/44 [00:01<00:00, 25.18it/s, loss=0.332, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 83:  91%|████████▏| 40/44 [00:01<00:00, 25.33it/s, loss=0.276, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 84:  91%|████████▏| 40/44 [00:01<00:00, 25.40it/s, loss=0.272, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 85:  91%|████████▏| 40/44 [00:01<00:00, 25.37it/s, loss=0.216, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 86:  91%|████████▏| 40/44 [00:01<00:00, 25.29it/s, loss=0.228, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 87:  91%|████████▏| 40/44 [00:01<00:00, 25.25it/s, loss=0.227, v_num=7lln]
Validation: 0it [00:00,

Epoch 92:  91%|█████████ | 40/44 [00:01<00:00, 25.24it/s, loss=0.22, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 93:  91%|████████▏| 40/44 [00:01<00:00, 25.49it/s, loss=0.179, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 94:  91%|████████▏| 40/44 [00:01<00:00, 25.45it/s, loss=0.185, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 95:  91%|█████████ | 40/44 [00:01<00:00, 25.44it/s, loss=0.17, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 96:  91%|█████████ | 40/44 [00:01<00:00, 25.45it/s, loss=0.18, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 97:  91%|████████▏| 40/44 [00:01<00:00, 25.48it/s, loss=0.177, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 98:  91%|████████▏| 40/44 [00:01<00:00, 25.45it/s, loss=0.155, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99:  91%|█████████ | 40/44 [00:01<00:00, 25.31it/s, loss=0.18, v_num=7lln]
Validation: 0it [00:00, ?it/s]
Epoch 99: 100%|██████████| 44/44 [00:01<00:00, 25.39it/s, loss=0.18, v_num=7lln]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 44/44 [00:01<00:00, 24.37it/s, loss=0.18, v_num=7lln]
